## <img src="./logo_UTN.svg" align="right" width="150" /> 
#### Procesamiento Digital de Señales

# Trabajo Práctico 3
#### Federico Gercensztein

***
## Estimación espectral


In [7]:
## Inicialización del Notebook del TP3

import numpy as np
import scipy.signal as sig
import matplotlib as mpl
import matplotlib.pyplot as plt
from pandas import DataFrame
from IPython.display import HTML
import scipy.signal as sig
from scipy.fftpack import fft, fftshift
from spectrum import CORRELOGRAMPSD
from spectrum.tools import cshift

fs = 1000 # Hz

<div class="alert alert-block alert-info">
<b>1)</b> Compruebe experimentalmente las propiedades de sesgo y varianza del periodograma.
</div>

<div class="alert alert-block alert-success">
<b>Ayuda:</b> Puede utilizar una señal aleatoria con valores normalmente distribuidos de media nula y varianza **NO** unitaria, es decir $ x \sim \mathcal{N}(\mu=0,\sigma^2=2)$

</div>

Es decir, que el periodograma es un estimador de la densidad de potencia espectral (Ver Hayes 8.2.2):

$$ \hat{P_P}(e^{\frac{2\pi·k·f_S}{N}}) = \hat{P_P}(k) = \frac{1}{N}· \lvert X(k) \rvert ^2  $$

 + **no sesgado asintóticamente** a medida que aumenta la ventana de registro N.
 + tiene varianza constante y **NO** depende de N.

In [3]:
# Simular para los siguientes tamaños de señal
N = np.array([10, 50, 100, 250, 500, 1000, 5000], dtype=np.float)

##########################################
# Acá podés generar los gráficos pedidos #
##########################################
u = 0
sigma_2 = 2

sesgos = np.zeros(len(N))
varianzas = np.zeros(len(N))
index = 0

for M in N:
       s = np.random.normal(u, np.sqrt(sigma_2), int(M))
       S = fft(s)
       modS = np.abs(S)
       periodgram = (modS**2)/M
       esperanza = np.mean(periodgram)
       sesgos[index] = esperanza
       varianzas[index] = np.var(periodgram)
       index+=1      

In [4]:

#######################################
# Tu simulación que genere resultados #
#######################################

tus_resultados_per = [ 
                   [sesgos[0], varianzas[0]], # <-- acá debería haber numeritos :)
                   [sesgos[1], varianzas[1]], # <-- acá debería haber numeritos :)
                   [sesgos[2], varianzas[2]], # <-- acá debería haber numeritos :)
                   [sesgos[3], varianzas[3]], # <-- acá debería haber numeritos :)
                   [sesgos[4], varianzas[4]], # <-- acá debería haber numeritos :)
                   [sesgos[5], varianzas[5]], # <-- acá debería haber numeritos :)
                   [sesgos[6], varianzas[6]], # <-- acá debería haber numeritos :)
                 ]
df = DataFrame(tus_resultados_per, columns=['$s_P$', '$v_P$'],
               index=N)
HTML(df.to_html())


,$s_P$,$v_P$
10.0,1.347162,3.526543
50.0,2.152480,4.416537
100.0,2.000833,4.016989
250.0,2.104431,4.413710
500.0,1.883034,3.097014
1000.0,2.031419,4.496370
5000.0,1.987834,3.948333


<div class="alert alert-block alert-info">
<b>2)</b>     Compruebe del mismo modo los resultados de sesgo y varianza vistos en la teoría del método de Bartlett.

</div>

Es decir, que el periodograma de Bartlett es un estimador de la densidad de potencia espectral que promedia K bloques disjuntos de las N muestras de una señal $x$ (Ver Hayes 8.2.4):

$$ \hat{P_B}(k) = \frac{1}{N}· \sum^{K-1}_{i=0} \lvert X(k) \rvert ^2  $$

 + **no sesgado asintóticamente** a medida que aumenta la ventana de registro N.
 + tiene varianza que decrece asintóticamente a medida que aumenta K.
 + tiene una resolución espectral K veces menor

In [5]:
# Simular para los siguientes tamaños de señal
N = 1000
K = np.array([2, 5, 10, 20, 50], dtype=np.float)

##########################################
# Acá podés generar los gráficos pedidos #
##########################################
u = 0
sigma_2 = 2
s = np.random.normal(u, np.sqrt(sigma_2), N)
sesgos = np.zeros(len(K))
varianzas = np.zeros(len(K))
index = 0

for k in K:
       L = int(N/k)
       S = np.transpose(np.vstack([fft(s[i*L:((i+1)*L)]) for i in range (0,int(k))]))
       modS = (np.abs(S)**2)/N
       periodgram = np.mean(modS, axis=1)
       esperanza = np.mean(periodgram)
       sesgos[index] = esperanza
       varianzas[index] = np.var(periodgram)
       index+=1

In [6]:

#######################################
# Tu simulación que genere resultados #
#######################################

tus_resultados_bartlett = [ 
                   [sesgos[0], varianzas[0]], # <-- acá debería haber numeritos :)
                   [sesgos[1], varianzas[1]], # <-- acá debería haber numeritos :)
                   [sesgos[2], varianzas[2]], # <-- acá debería haber numeritos :)
                   [sesgos[3], varianzas[3]], # <-- acá debería haber numeritos :)
                   [sesgos[4], varianzas[4]], # <-- acá debería haber numeritos :)
                 ]
df = DataFrame(tus_resultados_bartlett, columns=['$s_B$', '$v_B$'],
               index=K)
HTML(df.to_html())


,$s_B$,$v_B$
2.0,0.989600,0.495181
5.0,0.395840,0.030359
10.0,0.197920,0.004826
20.0,0.098960,0.000503
50.0,0.039584,0.000016


<div class="alert alert-block alert-info">
Estos resultados parecen no ser consistentes con la teoría del periodogroama de Bartlett. <br>
La varianza debería ser aproximadamente la varianza al cuadrado (4) sobre K.
</div>

<div class="alert alert-block alert-info">
<b>3)</b>     Compruebe del mismo modo los resultados de sesgo y varianza vistos en la teoría del método de Welch.

</div>

Es decir, que el periodograma de Welch es un estimador de la densidad de potencia espectral que promedia K bloques ventaneados por $w(n)$, posiblemente solapados de las N muestras de una señal $x$ (Ver Hayes 8.2.5):

$$ \hat{P_W}(k) = \frac{1}{K·L·U}· \sum^{K-1}_{i=0} \Bigg\vert \sum^{L-1}_{n=0}  x(n+i·D) · w(n) · e^{-j2\pi·k·n·\frac{f_S}{N}} \Bigg\vert^2   $$

 + **no sesgado asintóticamente** a medida que aumenta la ventana de registro N.
 + tiene varianza que decrece asintóticamente, a medida que se promedian más bloques de señal.
 + tiene una resolución espectral inversamente proporcional al tamaño del bloque.

In [9]:
# Simular para los siguientes tamaños de señal
N = 1000
slp = 50 # por ciento de ventanas adyacentes
K = np.array([2, 5, 10, 20, 50], dtype=np.float)

##########################################
# Acá podés generar los gráficos pedidos #
##########################################
u = 0
sigma_2 = 2
s = np.random.normal(u, np.sqrt(sigma_2), N)
sesgos = np.zeros(len(K))
varianzas = np.zeros(len(K))
index = 0

for k in K:
       L = int(N/k)
       f, S = sig.welch(s, nperseg=L) #hanning windowed, overlap default = 50%
       esperanza = np.mean(S)
       sesgos[index] = esperanza
       varianzas[index] = np.var(S)
       index+=1

In [16]:

#######################################
# Tu simulación que genere resultados #
#######################################

tus_resultados_welch = [ 
                   [sesgos[0], varianzas[0]], # <-- acá debería haber numeritos :)
                   [sesgos[1], varianzas[1]], # <-- acá debería haber numeritos :)
                   [sesgos[2], varianzas[2]], # <-- acá debería haber numeritos :)
                   [sesgos[3], varianzas[3]], # <-- acá debería haber numeritos :)
                   [sesgos[4], varianzas[4]], # <-- acá debería haber numeritos :)
                 ]
df = DataFrame(tus_resultados_welch, columns=['$s_B$', '$v_B$'],
               index=K)
HTML(df.to_html())


,$s_B$,$v_B$
2.0,4.123048,5.784472
5.0,4.128652,2.058581
10.0,4.099692,1.081507
20.0,3.917172,1.049927
50.0,3.606830,1.619639


<div class="alert alert-block alert-info">
<b>4)</b> Evalue el siguiente estimador de frecuencia de una senoidal contaminada por ruido incorrelado.

</div>

Para una señal $ x(k) = a_1 · \mathop{sen}(\Omega_1·k) + n(k)$

siendo 

  $\Omega_1 = \Omega_0 + f_r·\frac{2\pi}{N} $

  $\Omega_0 = \frac{\pi}{2} $
  
y las variables aleatorias definidas por

  $f_r \sim \mathcal{U}(-\frac{1}{2}, \, \frac{1}{2}) $

  $n \sim \mathcal{N}(0, \, \sigma ^2) $
  
Evalúe el siguiente estimador de $\Omega_1$

  $\hat{\Omega}_1^W = \mathop{arg\ max}_f \{ \hat{P_W} \} $
  
basado en el periodograma de Welch evaluado en **3)**. Del mismo modo, evalúe otro estimador de la PSD para crear otro estimador de $\Omega_1$

  $\hat{\Omega}_1^X = \mathop{arg\ max}_f \{ \hat{P_X} \} $

Considere 200 realizaciones de 1000 muestras para cada experimento. Cada realización debe tener un SNR tal que el pico de la senoidal esté 3 y 10 db por encima del *piso* de ruido impuesto por $n(k)$.


<div class="alert alert-block alert-success">
<b>Ayuda:</b> Puede utilizar el módulo de análisis espectral **Spectrum** donde encontrará la mayoría de los métodos descriptos en el Capítulo 8 del libro de Hayes.

</div>

In [8]:
# Simular para los siguientes tamaños de señal


N  = 1000 # muestras
fs = 1000 # Hz

k = 200 #realizaciones

fr = np.random.uniform(-0.5, 0.5, k)

f0 = fs/4 + fr*2*np.pi
p0 = 0     # radianes
a0db = 10
a0 = 10**(a0db/20)     # Volts

Ts = 1/fs
tt = np.linspace(0, (N-1)*Ts, N)

s = np.transpose(np.vstack([a0*np.sin(2*np.pi*ff*tt + p0) for ff in f0])) 

u = 0
sigma_2 = 2
dbDiff = 3 # Ruido 3dB por debajo del pico de la señal
dbSignal = 20*np.log10(a0)
an = 10**((dbSignal-dbDiff)/20)
n = an*np.transpose(np.vstack([np.random.normal(u, np.sqrt(sigma_2), N) for ff in f0]))

x = s + n

freqX, Xw = sig.welch(x, axis=0)

estimador = np.zeros(Xw.shape[0])
for r in range(0,Xw.shape[0]):
       estimador[r] = freqX[np.argmax(Xw[:,r])]
       
var = np.var(estimador)
estimador = np.mean(estimador)
estimador = estimador * fs


estimadorCorr = np.zeros(k)
for r in range(0,k):
       correlogram = CORRELOGRAMPSD(x[:,r], x[:,r], lag=300)
       correlogram = cshift(correlogram, len(correlogram)/2)
       correlogram = correlogram[int(correlogram.size/2): int(correlogram.size)]
       f = np.linspace(0, 0.5, int(len(correlogram)))
       estimadorCorr[r] = f[np.argmax(correlogram)]

varCorr = np.var(estimadorCorr)
mediaCorr = np.mean(estimadorCorr)
mediaCorr = mediaCorr * fs    

# Obtené los valores XX para que cumplas con el enunciado
#SNR = np.array([ XX, XX ], dtype=np.float)

##########################################
# Acá podés generar los gráficos pedidos #
##########################################


   a) ¿Qué estimador ha elegido? Explique brevemente los fundamentos principales y el enfoque del método elegido.


<div class="alert alert-block alert-warning">
<b>Respuesta:</b> Utilicé el correlograma (Blackman-Tuckey correlogram). El mismo utiliza la propiedades de la transformada de la autocorrelación para calcular la densidad espectral de potencia (PSD). Para que este método no tenga error se deberían tener infinitas muestras y obtener la autocorrelación de la señal para todo tiempo, en nuestro caso que tenemos la señal acotada sabemos que el método tendra cierto error de estimación (como todos los métodos).
</div>

   b) ¿Qué indicador considera que sería apropiado para poder comparar el rendimiento de ambos estimadores $i_j$?


<div class="alert alert-block alert-warning">
<b>Respuesta:</b> El indicador más importante para compara los métodos es la varianza, debido a que si el estimador tuviera una varianza baja pero estuviera en un valor equivocado, podemos entenderlo como un sesgo y modificar el estimador para que termine siendo insesgado, en cambio con la varianza no hay manera de eliminarla una vez definido el método.
</div>

In [10]:

#######################################
# Tu simulación que genere resultados #
#######################################
N  = 1000 # muestras
fs = 1000 # Hz

k = 200 #realizaciones

fr = np.random.uniform(-0.5, 0.5, k)

f0 = fs/4 + fr*2*np.pi
p0 = 0     # radianes
a0db = 10
a0 = 10**(a0db/20)     # Volts

Ts = 1/fs
tt = np.linspace(0, (N-1)*Ts, N)

s = np.transpose(np.vstack([a0*np.sin(2*np.pi*ff*tt + p0) for ff in f0])) 

u = 0
sigma_2 = 2
dbDiff = 3 # Ruido 3dB por debajo del pico de la señal
dbSignal = 20*np.log10(a0)
an = 10**((dbSignal-dbDiff)/20)
n = an*np.transpose(np.vstack([np.random.normal(u, np.sqrt(sigma_2), N) for ff in f0]))

x = s + n

freqX, Xw = sig.welch(x, axis=0)

estimador = np.zeros(Xw.shape[0])
for r in range(0,Xw.shape[0]):
       estimador[r] = freqX[np.argmax(Xw[:,r])]
       
varWelch10 = np.var(estimador)
meanWelch10 = np.mean(estimador)
meanWelch10 = meanWelch10 * fs


estimadorCorr = np.zeros(k)
for r in range(0,k):
       correlogram = CORRELOGRAMPSD(x[:,r], x[:,r], lag=100)
       correlogram = cshift(correlogram, len(correlogram)/2)
       correlogram = correlogram[int(correlogram.size/2): int(correlogram.size)]
       f = np.linspace(0, 0.5, int(len(correlogram)))
       estimadorCorr[r] = f[np.argmax(correlogram)]

varCorr10 = np.var(estimadorCorr)
meanCorr10 = np.mean(estimadorCorr)
meanCorr10 = meanCorr10 * fs    


N  = 1000 # muestras
fs = 1000 # Hz

k = 200 #realizaciones

fr = np.random.uniform(-0.5, 0.5, k)

f0 = fs/4 + fr*2*np.pi
p0 = 0     # radianes
a0db = 3
a0 = 10**(a0db/20)     # Volts

Ts = 1/fs
tt = np.linspace(0, (N-1)*Ts, N)

s = np.transpose(np.vstack([a0*np.sin(2*np.pi*ff*tt + p0) for ff in f0])) 

u = 0
sigma_2 = 2
dbDiff = 3 # Ruido 3dB por debajo del pico de la señal
dbSignal = 20*np.log10(a0)
an = 10**((dbSignal-dbDiff)/20)
n = an*np.transpose(np.vstack([np.random.normal(u, np.sqrt(sigma_2), N) for ff in f0]))

x = s + n

freqX, Xw = sig.welch(x, axis=0)

estimador = np.zeros(Xw.shape[0])
for r in range(0,Xw.shape[0]):
       estimador[r] = freqX[np.argmax(Xw[:,r])]
       
varWelch3 = np.var(estimador)
meanWelch3 = np.mean(estimador)
meanWelch3 = meanWelch3 * fs


estimadorCorr = np.zeros(k)
for r in range(0,k):
       correlogram = CORRELOGRAMPSD(x[:,r], x[:,r], lag=100)
       correlogram = cshift(correlogram, len(correlogram)/2)
       correlogram = correlogram[int(correlogram.size/2): int(correlogram.size)]
       f = np.linspace(0, 0.5, int(len(correlogram)))
       estimadorCorr[r] = f[np.argmax(correlogram)]

varCorr3 = np.var(estimadorCorr)
meanCorr3 = np.mean(estimadorCorr)
meanCorr3 = meanCorr3 * fs    



# Una vez definido tu indicador de performance, calculalo y comparalo para las situaciones pedidas.
tus_resultados = [ 
                   [varWelch3, varCorr3], # <-- acá debería haber numeritos :)
                   [varWelch10, varCorr10] # <-- acá debería haber numeritos :)
                 ]
df = DataFrame(tus_resultados, columns=['$i_W$', '$i_X$'],
               index=[  
                        '3 dB',
                        '10 dB'
                     ])
HTML(df.to_html())


,$i_W$,$i_X$
3 dB,0.000006,0.000003
10 dB,0.000005,0.000003
